## Check qualities after demultiplex

In [ ]:
logdir=/workspace/hramzr/2_Phd_PROJECT/GBS/indv_snapper_purged/logs
snapper_dir=/workspace/hramzr/2_Phd_PROJECT/GBS/indv_snapper_purged/

In [ ]:
module load FastQC/0.11.7
for file in $(ls $snapper_dir | egrep ".fq")
do
bsub \
-e ${snapper_dir}logs/FASTQC.e -o ${snapper_dir}logs/FASTQC.o -J "FQC" \
"fastqc -t 5 ${snapper_dir}${file} -o ${snapper_dir}FASTQC/"
done

# trimming

In [ ]:
#make list with filenames
ls ${snapper_dir} | egrep "*gz" | tr -d [a-z.] \
>/workspace/hramzr/2_Phd_PROJECT/GBS/indv_snapper_purged/tmp/files

In [ ]:
TRPATH=/software/bioinformatics/Trimmomatic-0.36
# cat ${TRPATH}/adapters/TruSeq2-SE.fa

cat ${TRPATH}/adapters/TruSeq3-SE.fa

In [ ]:
TRPATH=/software/bioinformatics/Trimmomatic-0.36
trimmed_path=/workspace/hramzr/2_Phd_PROJECT/GBS/trimmed_purged/
snapper_dir=/workspace/hramzr/2_Phd_PROJECT/GBS/indv_snapper_purged/
tlog=/workspace/hramzr/2_Phd_PROJECT/GBS/trimming_log/
while read -r filename
do
bsub \
-o ${tlog}trim.o -e${tlog}trim.e -J "trimmyboi" \
"java -jar ${TRPATH}/trimmomatic-0.36.jar \
SE -phred33 \
${snapper_dir}${filename}.fq.gz \
${trimmed_path}${filename}_trimmed.fq.gz \
ILLUMINACLIP:${TRPATH}/adapters/TruSeq2-SE.fa:2:30:10 \
SLIDINGWINDOW:4:20 \
MINLEN:50"
done < /workspace/hramzr/2_Phd_PROJECT/GBS/indv_snapper_purged/tmp/files

# Check qualities after trim

In [ ]:
module load FastQC/0.11.7
fqcdir=/workspace/hramzr/2_Phd_PROJECT/GBS/trimmed_purged/FASTQC_log/
bsub \
-e ${fqcdir}FASTQC.e -o ${fqcdir}FASTQC.o -J "FQC" \
"fastqc -t 5 ${trimmed_path}*.gz -o /workspace/hramzr/2_Phd_PROJECT/GBS/trimmed_purged/FASTQC"

# index ref genome

In [ ]:
#copy over the genome
genome=/workspace/hramzr/2_Phd_PROJECT/snapper_genome/Chrysophrys_auratus.v.1.0.all.male.map.fasta
genomedir=/workspace/hramzr/2_Phd_PROJECT/snapper_genome/
genome2=/powerplant/output/genomic/fish/Chrysophrys/auratus/Genome/1000.Genome.v.1.0/Chrysophrys_auratus.v.1.0.chromosomes.male.map.fsa
bsub \
-e ${genomedir}cpgenome.e -o ${genomedir}cpgenome.o -J "cp sn genome" \
"cp $genome2 /workspace/hramzr/2_Phd_PROJECT/snapper_genome/"


In [ ]:
#index it

genome2=/workspace/hramzr/2_Phd_PROJECT/snapper_genome/Chrysophrys_auratus.v.1.0.chromosomes.male.map.fsa
module load samtools/1.7
module load bwa/0.7.17
bsub \
-e ${genomedir}indexgenome.e -o ${genomedir}indexgenome.o -J "index genome" \
"bwa index $genome2"

# Alignment

In [ ]:
#get filenames to be aligned
ls GBS/trimmed_purged | egrep "fq" | awk -F "_" '{print $0, $1}' \
>/workspace/hramzr/2_Phd_PROJECT/GBS/indv_snapper_purged/tmp/trimmed_files

In [41]:
module load samtools/1.7
module load bwa/0.7.17
outdir=/workspace/hramzr/2_Phd_PROJECT/GBS/bamfiles_purged/
log=/workspace/hramzr/2_Phd_PROJECT/GBS/bamfiles_purged/logs/
indir=/workspace/hramzr/2_Phd_PROJECT/GBS/trimmed_purged/
genome=/workspace/hramzr/2_Phd_PROJECT/snapper_genome/Chrysophrys_auratus.v.1.0.all.male.map.fasta
genome2=/workspace/hramzr/2_Phd_PROJECT/snapper_genome/Chrysophrys_auratus.v.1.0.chromosomes.male.map.fsa

while read -r fn id
do
    bsub \
    -o ${log}align.o -e ${log}align.e -J "align ${id}" \
    "bwa mem -t 8  \
    ${genome2} ${indir}${fn} \
    | samtools view -Su - | samtools sort - -o ${outdir}${id}.bam"
done < /workspace/hramzr/2_Phd_PROJECT/GBS/indv_snapper_purged/tmp/trimmed_files

Job <344844> is submitted to default queue <lowpriority>.
Job <344845> is submitted to default queue <lowpriority>.
Job <344846> is submitted to default queue <lowpriority>.
Job <344847> is submitted to default queue <lowpriority>.
Job <344848> is submitted to default queue <lowpriority>.
Job <344849> is submitted to default queue <lowpriority>.
Job <344850> is submitted to default queue <lowpriority>.
Job <344851> is submitted to default queue <lowpriority>.
Job <344852> is submitted to default queue <lowpriority>.
Job <344853> is submitted to default queue <lowpriority>.
Job <344854> is submitted to default queue <lowpriority>.
Job <344855> is submitted to default queue <lowpriority>.
Job <344856> is submitted to default queue <lowpriority>.
Job <344857> is submitted to default queue <lowpriority>.
Job <344858> is submitted to default queue <lowpriority>.
Job <344859> is submitted to default queue <lowpriority>.
Job <344860> is submitted to default queue <lowpriority>.
Job <344861> i

In [ ]:
module load FastQC/0.11.7
bdir=/workspace/hramzr/2_Phd_PROJECT/GBS/bamfiles_purged/
fqdir=/workspace/hramzr/2_Phd_PROJECT/GBS/bamfiles_purged/FASTQC/
logdir=/powerplant/workspace/hramzr/2_Phd_PROJECT/GBS/bamfiles_purged/logs
bsub \
-e ${logdir}FASTQC.e -o ${logdir}FASTQC.o -J "FQC" \
"fastqc -t 5 ${bdir}*.bam -o ${fqdir}"

# Remove duplicates

In [ ]:
#make list of bam file names
ls /powerplant/workspace/hramzr/2_Phd_PROJECT/GBS/bamfiles_purged/ | egrep "*.bam" | tr -d [a-z.] \
>/powerplant/workspace/hramzr/2_Phd_PROJECT/GBS/tmp/bamlist

In [118]:
bdir=/powerplant/workspace/hramzr/2_Phd_PROJECT/GBS/bamfiles_purged/
outdir=/powerplant/workspace/hramzr/2_Phd_PROJECT/GBS/bamfiles_purged_nodups/
mdir=/powerplant/workspace/hramzr/2_Phd_PROJECT/GBS/bamfiles_purged_nodups/marked/
tmpdir=/powerplant/workspace/hramzr/2_Phd_PROJECT/GBS/bamfiles_purged_nodups/tmp
log=/powerplant/workspace/hramzr/2_Phd_PROJECT/GBS/bamfiles_purged_nodups/log/
PICARD=/software/bioinformatics/picard-tools-2.18.7/picard.jar
while read -r id
do
    bsub  \
    -o ${log}dup.o -e ${log}dup.e -J "mdup ${id}" \
    -R "rusage[mem=640000] span[hosts=1]" \
    "java -jar -Xmx32G -Djava.io.tmpdir=${tmpdir} $PICARD MarkDuplicates \
    MAX_RECORDS_IN_RAM=500000 \
    INPUT=${bdir}${id}.bam \
    OUTPUT=${outdir}${id}.bam \
    AS=true \
    M=${mdir}${id}.txt \
    TMP_DIR=${tmpdir} \
    REMOVE_DUPLICATES=true"
done < /powerplant/workspace/hramzr/2_Phd_PROJECT/GBS/tmp/bamlist

Job <333129> is submitted to default queue <lowpriority>.
Job <333130> is submitted to default queue <lowpriority>.
Job <333131> is submitted to default queue <lowpriority>.
Job <333132> is submitted to default queue <lowpriority>.
Job <333133> is submitted to default queue <lowpriority>.
Job <333134> is submitted to default queue <lowpriority>.
Job <333135> is submitted to default queue <lowpriority>.
Job <333136> is submitted to default queue <lowpriority>.
Job <333137> is submitted to default queue <lowpriority>.
Job <333138> is submitted to default queue <lowpriority>.
Job <333139> is submitted to default queue <lowpriority>.
Job <333140> is submitted to default queue <lowpriority>.
Job <333141> is submitted to default queue <lowpriority>.
Job <333142> is submitted to default queue <lowpriority>.
Job <333143> is submitted to default queue <lowpriority>.
Job <333144> is submitted to default queue <lowpriority>.
Job <333145> is submitted to default queue <lowpriority>.
Job <333146> i

# Check quality after removing duplicates

In [ ]:
module load FastQC/0.11.7
bdir=/powerplant/workspace/hramzr/2_Phd_PROJECT/GBS/bamfiles_purged_nodups/
fqdir=/powerplant/workspace/hramzr/2_Phd_PROJECT/GBS/bamfiles_purged_nodups/FASTQC/
logdir=/powerplant/workspace/hramzr/2_Phd_PROJECT/GBS/bamfiles_purged_nodups/log/

bsub \
-e ${logdir}FASTQC.e -o ${logdir}FASTQC.o -J "FQC" \
"fastqc -t 5 ${bdir}*.bam -o ${fqdir}"

# Add read groups for generating VCF

In [ ]:
cat /powerplant/workspace/hramzr/2_Phd_PROJECT/GBS/tmp/bamlist

In [154]:
module load samtools/1.7
PICARD=/software/bioinformatics/picard-tools-2.18.7/picard.jar
indir=/powerplant/workspace/hramzr/2_Phd_PROJECT/GBS/bamfiles_purged_nodups/
outdir=/workspace/hramzr/2_Phd_PROJECT/GBS/bamfiles_nodupsrg/
tmp=/workspace/hramzr/2_Phd_PROJECT/GBS/bamfiles_nodupsrg/tmp/
log=/workspace/hramzr/2_Phd_PROJECT/GBS/bamfiles_nodupsrg/log/

while read -r id
do
    bsub \
    -o ${log}rg.o -e ${log}rg.e -J "rg ${id}" \
    "java -jar -Xmx32G -Djava.io.tmpdir=${tmp} $PICARD AddOrReplaceReadGroups \
    MAX_RECORDS_IN_RAM=500000 \
    I=${indir}${id}.bam \
    O=${outdir}${id}.bam \
    RGLB=lib1 \
    RGPL=illumina \
    RGPU=unit1 \
    RGID=${id} \
    RGSM=${id}"
done < /powerplant/workspace/hramzr/2_Phd_PROJECT/GBS/tmp/bamlist
    


Job <335926> is submitted to default queue <lowpriority>.
Job <335927> is submitted to default queue <lowpriority>.
Job <335928> is submitted to default queue <lowpriority>.
Job <335929> is submitted to default queue <lowpriority>.
Job <335930> is submitted to default queue <lowpriority>.
Job <335931> is submitted to default queue <lowpriority>.
Job <335932> is submitted to default queue <lowpriority>.
Job <335933> is submitted to default queue <lowpriority>.
Job <335934> is submitted to default queue <lowpriority>.
Job <335935> is submitted to default queue <lowpriority>.
Job <335936> is submitted to default queue <lowpriority>.
Job <335937> is submitted to default queue <lowpriority>.
Job <335938> is submitted to default queue <lowpriority>.
Job <335939> is submitted to default queue <lowpriority>.
Job <335940> is submitted to default queue <lowpriority>.
Job <335941> is submitted to default queue <lowpriority>.
Job <335942> is submitted to default queue <lowpriority>.
Job <335943> i

# add read groups nondeduplicated

In [83]:
module load samtools/1.7
PICARD=/software/bioinformatics/picard-tools-2.18.7/picard.jar
indir=/workspace/hramzr/2_Phd_PROJECT/GBS/bamfiles_purged/
outdir=/workspace/hramzr/2_Phd_PROJECT/GBS/bamfiles_markedrg/
tmp=/workspace/hramzr/2_Phd_PROJECT/GBS/bamfiles_markedrg/tmp
log=/workspace/hramzr/2_Phd_PROJECT/GBS/bamfiles_markedrg/log/
while read -r id
do
    bsub \
    -o ${log}rg.o -e ${log}rg.e -J "rg ${id}" \
    "java -jar -Xmx32G -Djava.io.tmpdir=${tmp} $PICARD AddOrReplaceReadGroups \
    MAX_RECORDS_IN_RAM=500000 \
    I=${indir}${id}.bam \
    O=${outdir}${id}.bam \
    RGLB=lib1 \
    RGPL=illumina \
    RGPU=unit1 \
    RGID=${id} \
    RGSM=${id}"
done < /powerplant/workspace/hramzr/2_Phd_PROJECT/GBS/tmp/bamlist
    


Job <378723> is submitted to default queue <lowpriority>.
Job <378724> is submitted to default queue <lowpriority>.
Job <378725> is submitted to default queue <lowpriority>.
Job <378726> is submitted to default queue <lowpriority>.
Job <378727> is submitted to default queue <lowpriority>.
Job <378728> is submitted to default queue <lowpriority>.
Job <378729> is submitted to default queue <lowpriority>.
Job <378730> is submitted to default queue <lowpriority>.
Job <378731> is submitted to default queue <lowpriority>.
Job <378732> is submitted to default queue <lowpriority>.
Job <378733> is submitted to default queue <lowpriority>.
Job <378734> is submitted to default queue <lowpriority>.
Job <378735> is submitted to default queue <lowpriority>.
Job <378736> is submitted to default queue <lowpriority>.
Job <378737> is submitted to default queue <lowpriority>.
Job <378738> is submitted to default queue <lowpriority>.
Job <378739> is submitted to default queue <lowpriority>.
Job <378740> i

# Marked duplicates

In [85]:
bdir=/powerplant/workspace/hramzr/2_Phd_PROJECT/GBS/bamfiles_markedrg/
outdir=/workspace/hramzr/2_Phd_PROJECT/GBS/bamfiles_markedrg/marked_bams/
mdir=${outdir}marked/
tmpdir=${outdir}tmp
log=${outdir}log/
PICARD=/software/bioinformatics/picard-tools-2.18.7/picard.jar
while read -r id
do
    bsub  \
    -o ${log}dup.o -e ${log}dup.e -J "mdup ${id}" \
    -R "rusage[mem=16400] span[hosts=1]" \
    "java -jar -Xmx32G -Djava.io.tmpdir=${tmpdir} $PICARD MarkDuplicates \
    MAX_RECORDS_IN_RAM=500000 \
    INPUT=${bdir}${id}.bam \
    OUTPUT=${outdir}${id}.bam \
    AS=true \
    M=${mdir}${id}.txt \
    TMP_DIR=${tmpdir}"
done < /powerplant/workspace/hramzr/2_Phd_PROJECT/GBS/tmp/bamlist

Job <380064> is submitted to default queue <lowpriority>.
Job <380065> is submitted to default queue <lowpriority>.
Job <380066> is submitted to default queue <lowpriority>.
Job <380067> is submitted to default queue <lowpriority>.
Job <380068> is submitted to default queue <lowpriority>.
Job <380069> is submitted to default queue <lowpriority>.
Job <380070> is submitted to default queue <lowpriority>.
Job <380071> is submitted to default queue <lowpriority>.
Job <380072> is submitted to default queue <lowpriority>.
Job <380073> is submitted to default queue <lowpriority>.
Job <380074> is submitted to default queue <lowpriority>.
Job <380075> is submitted to default queue <lowpriority>.
Job <380076> is submitted to default queue <lowpriority>.
Job <380077> is submitted to default queue <lowpriority>.
Job <380078> is submitted to default queue <lowpriority>.
Job <380079> is submitted to default queue <lowpriority>.
Job <380080> is submitted to default queue <lowpriority>.
Job <380081> i

In [60]:
cat /powerplant/workspace/hramzr/2_Phd_PROJECT/GBS/tmp/bamlist | wc -l

661


In [162]:
module load FastQC/0.11.7
bdir=/workspace/hramzr/2_Phd_PROJECT/GBS/bamfiles_nodupsrg/
fqdir=/workspace/hramzr/2_Phd_PROJECT/GBS/bamfiles_nodupsrg/FASTQC/
logdir=/workspace/hramzr/2_Phd_PROJECT/GBS/bamfiles_nodupsrg/log/

bsub \
-e ${logdir}FASTQC.e -o ${logdir}FASTQC.o -J "FQC" \
"fastqc -t 5 ${bdir}*.bam -o ${fqdir}"

Job <337286> is submitted to default queue <lowpriority>.


# Index for further processes

In [1]:
module load samtools/1.7
log=/workspace/hramzr/2_Phd_PROJECT/GBS/bamfiles_purged/logs/
indir=/workspace/hramzr/2_Phd_PROJECT/GBS/bamfiles_purged/

while read -r id
do
    bsub \
    -o ${log}ind.o -e ${log}ind.e -J "index ${id}" \
    "samtools index ${indir}${id}.bam"
done < /powerplant/workspace/hramzr/2_Phd_PROJECT/GBS/tmp/bamlist

Job <414835> is submitted to default queue <lowpriority>.
Job <414836> is submitted to default queue <lowpriority>.
Job <414837> is submitted to default queue <lowpriority>.
Job <414838> is submitted to default queue <lowpriority>.
Job <414839> is submitted to default queue <lowpriority>.
Job <414840> is submitted to default queue <lowpriority>.
Job <414841> is submitted to default queue <lowpriority>.
Job <414842> is submitted to default queue <lowpriority>.
Job <414843> is submitted to default queue <lowpriority>.
Job <414844> is submitted to default queue <lowpriority>.
Job <414845> is submitted to default queue <lowpriority>.
Job <414846> is submitted to default queue <lowpriority>.
Job <414847> is submitted to default queue <lowpriority>.
Job <414848> is submitted to default queue <lowpriority>.
Job <414849> is submitted to default queue <lowpriority>.
Job <414850> is submitted to default queue <lowpriority>.
Job <414851> is submitted to default queue <lowpriority>.
Job <414852> i

Job <414977> is submitted to default queue <lowpriority>.
Job <414978> is submitted to default queue <lowpriority>.
Job <414979> is submitted to default queue <lowpriority>.
Job <414980> is submitted to default queue <lowpriority>.
Job <414981> is submitted to default queue <lowpriority>.
Job <414982> is submitted to default queue <lowpriority>.
Job <414983> is submitted to default queue <lowpriority>.
Job <414984> is submitted to default queue <lowpriority>.
Job <414985> is submitted to default queue <lowpriority>.
Job <414986> is submitted to default queue <lowpriority>.
Job <414987> is submitted to default queue <lowpriority>.
Job <414988> is submitted to default queue <lowpriority>.
Job <414989> is submitted to default queue <lowpriority>.
Job <414990> is submitted to default queue <lowpriority>.
Job <414991> is submitted to default queue <lowpriority>.
Job <414992> is submitted to default queue <lowpriority>.
Job <414993> is submitted to default queue <lowpriority>.
Job <414994> i

Job <415119> is submitted to default queue <lowpriority>.
Job <415120> is submitted to default queue <lowpriority>.
Job <415121> is submitted to default queue <lowpriority>.
Job <415122> is submitted to default queue <lowpriority>.
Job <415123> is submitted to default queue <lowpriority>.
Job <415124> is submitted to default queue <lowpriority>.
Job <415125> is submitted to default queue <lowpriority>.
Job <415126> is submitted to default queue <lowpriority>.
Job <415127> is submitted to default queue <lowpriority>.
Job <415128> is submitted to default queue <lowpriority>.
Job <415129> is submitted to default queue <lowpriority>.
Job <415130> is submitted to default queue <lowpriority>.
Job <415131> is submitted to default queue <lowpriority>.
Job <415132> is submitted to default queue <lowpriority>.
Job <415133> is submitted to default queue <lowpriority>.
Job <415134> is submitted to default queue <lowpriority>.
Job <415135> is submitted to default queue <lowpriority>.
Job <415136> i

Job <415261> is submitted to default queue <lowpriority>.
Job <415262> is submitted to default queue <lowpriority>.
Job <415263> is submitted to default queue <lowpriority>.
Job <415264> is submitted to default queue <lowpriority>.
Job <415265> is submitted to default queue <lowpriority>.
Job <415266> is submitted to default queue <lowpriority>.
Job <415267> is submitted to default queue <lowpriority>.
Job <415268> is submitted to default queue <lowpriority>.
Job <415269> is submitted to default queue <lowpriority>.
Job <415270> is submitted to default queue <lowpriority>.
Job <415271> is submitted to default queue <lowpriority>.
Job <415272> is submitted to default queue <lowpriority>.
Job <415273> is submitted to default queue <lowpriority>.
Job <415274> is submitted to default queue <lowpriority>.
Job <415275> is submitted to default queue <lowpriority>.
Job <415276> is submitted to default queue <lowpriority>.
Job <415277> is submitted to default queue <lowpriority>.
Job <415278> i

Job <415403> is submitted to default queue <lowpriority>.
Job <415404> is submitted to default queue <lowpriority>.
Job <415405> is submitted to default queue <lowpriority>.
Job <415406> is submitted to default queue <lowpriority>.
Job <415407> is submitted to default queue <lowpriority>.
Job <415408> is submitted to default queue <lowpriority>.
Job <415409> is submitted to default queue <lowpriority>.
Job <415410> is submitted to default queue <lowpriority>.
Job <415411> is submitted to default queue <lowpriority>.
Job <415412> is submitted to default queue <lowpriority>.
Job <415413> is submitted to default queue <lowpriority>.
Job <415414> is submitted to default queue <lowpriority>.
Job <415415> is submitted to default queue <lowpriority>.
Job <415416> is submitted to default queue <lowpriority>.
Job <415417> is submitted to default queue <lowpriority>.
Job <415418> is submitted to default queue <lowpriority>.
Job <415419> is submitted to default queue <lowpriority>.
Job <415420> i

# VCF generation

In [18]:
#index as is needed for freebayes
module load samtools/1.7
genome2=/workspace/hramzr/2_Phd_PROJECT/snapper_genome/Chrysophrys_auratus.v.1.0.chromosomes.male.map.fsa
genome=/workspace/hramzr/2_Phd_PROJECT/snapper_genome/Chrysophrys_auratus.v.1.0.all.male.map.fasta
samtools faidx $genome2

In [101]:
#list with dirs and files of bams

find /workspace/hramzr/2_Phd_PROJECT/GBS/bamfiles_markedrg/*.bam>/powerplant/workspace/hramzr/2_Phd_PROJECT/GBS/tmp/vcfbams_dups
egrep -f /workspace/hramzr/2_Phd_PROJECT/phenotypic_data/present_ids \
/powerplant/workspace/hramzr/2_Phd_PROJECT/GBS/tmp/vcfbams_dups>/powerplant/workspace/hramzr/2_Phd_PROJECT/GBS/tmp/vcfbams_dups_infeno

In [156]:
find /workspace/hramzr/2_Phd_PROJECT/GBS/bamfiles_nodupsrg/*.bam>/powerplant/workspace/hramzr/2_Phd_PROJECT/GBS/tmp/vcfbamsdups
egrep -f /workspace/hramzr/2_Phd_PROJECT/phenotypic_data/present_ids \
/powerplant/workspace/hramzr/2_Phd_PROJECT/GBS/tmp/vcfbamsdups>/powerplant/workspace/hramzr/2_Phd_PROJECT/GBS/tmp/vcfbamsdups_infeno

In [103]:
cat /powerplant/workspace/hramzr/2_Phd_PROJECT/GBS/tmp/vcfbams_dups_infeno

/workspace/hramzr/2_Phd_PROJECT/GBS/bamfiles_markedrg/101870070.bam
/workspace/hramzr/2_Phd_PROJECT/GBS/bamfiles_markedrg/105105149.bam
/workspace/hramzr/2_Phd_PROJECT/GBS/bamfiles_markedrg/109947113.bam
/workspace/hramzr/2_Phd_PROJECT/GBS/bamfiles_markedrg/110127331.bam
/workspace/hramzr/2_Phd_PROJECT/GBS/bamfiles_markedrg/112275849.bam
/workspace/hramzr/2_Phd_PROJECT/GBS/bamfiles_markedrg/115714093.bam
/workspace/hramzr/2_Phd_PROJECT/GBS/bamfiles_markedrg/118473381.bam
/workspace/hramzr/2_Phd_PROJECT/GBS/bamfiles_markedrg/118630322.bam
/workspace/hramzr/2_Phd_PROJECT/GBS/bamfiles_markedrg/118917609.bam
/workspace/hramzr/2_Phd_PROJECT/GBS/bamfiles_markedrg/120986715.bam
/workspace/hramzr/2_Phd_PROJECT/GBS/bamfiles_markedrg/121310723.bam
/workspace/hramzr/2_Phd_PROJECT/GBS/bamfiles_markedrg/121437381.bam
/workspace/hramzr/2_Phd_PROJECT/GBS/bamfiles_markedrg/122410273.bam
/workspace/hramzr/2_Phd_PROJECT/GBS/bamfiles_markedrg/123064907.bam
/workspace/hramzr/2_Phd_PROJECT/GBS/bamfiles_mar

In [21]:
module load samtools/1.9
samtools idxstats /workspace/hramzr/2_Phd_PROJECT/GBS/bamfiles_markedrg/123064907.bam


samtools/1.9(23):ERROR:150: Module 'samtools/1.9' conflicts with the currently loaded module(s) 'samtools/1.7'
samtools/1.9(23):ERROR:102: Tcl command execution failed: conflict ${appname}

LG1	35796517	84250	0
LG2	36161541	87391	0
LG3	34661116	80553	0
LG4	33120650	82666	0
LG5	35422278	82920	0
LG6	28599003	75822	0
LG7	34240341	85110	0
LG8	37046140	82341	0
LG9	27277866	74500	0
LG10	33112362	83488	0
LG11	38628475	87027	0
LG12	26212225	74573	0
LG13	33878147	77390	0
LG14	29913436	74125	0
LG15	28243467	73305	0
LG16	30315390	73461	0
LG17	24333991	67498	0
LG18	32765680	78359	0
LG19	28261070	73086	0
LG20	26584312	67043	0
LG21	28350763	73145	0
LG22	29453248	80352	0
LG23	20385525	51541	0
LG24	17221635	47255	0
LG25	3820059	11599	0
*	0	0	135920


In [104]:
export TMPDIR=/workspace/hramzr/2_Phd_PROJECT/GBS/vcf_snapper/tmp
module load freebayes/1.1.0
module load tabix/0.2.6
module load samtools/1.7


LOG=/workspace/hramzr/2_Phd_PROJECT/GBS/vcf_snapper_dups/log/
REF2=/workspace/hramzr/2_Phd_PROJECT/snapper_genome/Chrysophrys_auratus.v.1.0.chromosomes.male.map.fsa
REF=/workspace/hramzr/2_Phd_PROJECT/snapper_genome/Chrysophrys_auratus.v.1.0.all.male.map.fasta
BAMLST=/powerplant/workspace/hramzr/2_Phd_PROJECT/GBS/tmp/vcfbams_dups_infeno
REFINDEX=${REF2}.fai
PAR=/software/bioinformatics/freebayes-1.1.0/scripts

bsub << EOF
#!/bin/bash
#BSUB -J "Free"
#BSUB -o ${LOG}freeb.out 
#BSUB -e ${LOG}freeb.err
#BSUB -n 20
#BSUB -R "rusage[mem=2280000] span[hosts=1]"

freebayes-parallel <($PAR/fasta_generate_regions.py $REFINDEX 100000) 60 \
-f $REF2 -L $BAMLST -C 5 -k --report-genotype-likelihood-max --min-mapping-quality 10 \
--genotype-qualities --use-mapping-quality --no-mnps --no-complex> /workspace/hramzr/2_Phd_PROJECT/GBS/vcf_snapper_dups/630_snappers_dupsnchrom.vcf
EOF

Job <387541> is submitted to default queue <lowpriority>.


In [22]:
module avail


---------- /software/OSutils/modules-4.4.1/share/Modules/modulefiles -----------
1SynChro/1.15
3d-dna/180922
A5-miseq/20160825
abyss/2.0.2
abyss/2.1.0
abyss/2.1.1
abyss/2.1.2
abyss/2.1.3
abyss/2.1.4
abyss/2.1.5
adapterremoval/2.2.2
AfterQC/0.9.1
AfterQC/0.9.5
allpathslg/44837
allpathslg/44913
allpathslg/49722
allpathslg/50191
allpathslg/52488
amos/3.1.0
angsd/0.913
angsd/0.917
angsd/0.917-116-g5d087b2
angsd/0.918
angsd/0.919
angsd/0.921
angsd/0.922
angsd/0.923
angsd/0.925
angsd-wrapper/7224db2
anytag/2.5.2
apsim/7.10-r49ace54f9c8a670190aef9d8d0fb9d5477bb1534
apsim/7.9-r4047
apsimx/2018.01.30.2253
apsimx/2018.09.28.3099
apsimx/2019.01.08.3392
apsimx/2019.04.03.3693
apsimx/2019.06.05.3920
apsimx/2019.07.18.4025
apsimx/2019.10.04.4236
apsimx/2020.04.09.5012
apsimx/2020.08.04.5350
arcs/1.0.6
art/MountRainier
aspera-cli/3.7.2.354.010c3b8
assemblathon_stats/14dfdab
assemblathon_stats/2011_10_13
asub/2.1
augustus/3.2.2
augustus/3.3
augustus/3.3.1
augustus/3.3.3-new
aws-cli/2.0.43
bam2fastx/1

# VCF stats

In [100]:
module load rtg-tools/3.7.1-eb13bbb
stats_dir=/workspace/hramzr/2_Phd_PROJECT/GBS/vcf_snapper_dups/stats/
vcf=/workspace/hramzr/2_Phd_PROJECT/GBS/vcf_snapper_dups/630_snappers_dupsnchrom.vcf

bsub \
-e ${statsdir}ste -o ${statsdir}sto -J "stats" \
"rtg vcfstats $vcf>${stats_dir}snps_dups.txt"


Job <385323> is submitted to default queue <lowpriority>.


In [79]:
cat 630_snappers_dupsnchrom.vcf | egrep "CHROM"

#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO	FORMAT	unknown


In [168]:
cat /workspace/hramzr/2_Phd_PROJECT/GBS/vcf_snapper_dups/stats/snps_nodups.txt | egrep "Sample Name" -A1  | tr -d "\n" | tr "-" "\n" | \
tr -d " " | tr -d [A-Za-z] | awk -F ":" '{print $0}' | sed -r 's/^.{1}//' | tr ":" " " | sed -r '/^\s*$/d' \
>/powerplant/workspace/hramzr/2_Phd_PROJECT/GBS/vcf_snapper_dups/SNPS_per_indv

In [29]:
cat

snps_nodups.txt  snps.txt


In [51]:
cat snps_nodups.txt 
# | awk -F ":" '{print $2}' | sort -n

Location                     : /workspace/hramzr/2_Phd_PROJECT/GBS/vcf_snapper_dups/630_snappers_nodupsnchrom.vcf
Failed Filters               : 0
Passed Filters               : 2362

Sample Name: 771402329
SNPs                         : 1274
MNPs                         : 0
Insertions                   : 16
Deletions                    : 20
Indels                       : 0
Same as reference            : 473
Missing Genotype             : 579
SNP Transitions/Transversions: 1.26 (1268/1003)
Total Het/Hom ratio          : 0.29 (294/1016)
SNP Het/Hom ratio            : 0.29 (287/987)
MNP Het/Hom ratio            : - (0/0)
Insertion Het/Hom ratio      : 0.14 (2/14)
Deletion Het/Hom ratio       : 0.33 (5/15)
Indel Het/Hom ratio          : - (0/0)
Insertion/Deletion ratio     : 0.80 (16/20)
Indel/SNP+MNP ratio          : 0.03 (36/1274)

Sample Name: 599220335
SNPs                         : 1346
MNPs                         : 0
Insertions                   : 16
Deletions                    : 

In [113]:
for val in {1..8}
do
    bsub -w "started(${val}-1)" -J "$val" \
    "echo $val>>ktest"
done


1-1: No matching job found. Job not submitted.
2-1: No matching job found. Job not submitted.
3-1: No matching job found. Job not submitted.
4-1: No matching job found. Job not submitted.
5-1: No matching job found. Job not submitted.
6-1: No matching job found. Job not submitted.
7-1: No matching job found. Job not submitted.
8-1: No matching job found. Job not submitted.


: 255

In [125]:
while read -r id snps
do 
/input/genomic/fish/Chrysophrys/auratus/Resequencing/CAGRF15217/sample_and_individual_identification.txt
done < /powerplant/workspace/hramzr/2_Phd_PROJECT/GBS/vcf_snapper_dups/SNPS_per_indv 

239324475 70721
399847148 46159
129715486 74227
8844351 73505
74110447 69487
546467689 67811
605449782 72908
477892890 67982
534746604 71715
664894268 67110
946380271 69720
616591985 72450
812247317 71360
262447031 71271
959622330 73134
260457774 69807
824159252 72595
636039763 70972
729784720 75277
275360367 70978
578471501 69095
966195161 73922
8826052 74407
557159709 74091
959685696 74995
593795246 69476
309014110 71528
299826357 71823
8835264 72720
252462567 73791
664735520 72178
776448125 65363
650619794 71129
960458269 65471
929648027 67515
241682971 72876
741401117 72896
123064907 68694
689853268 74204
9075102 68271
574110594 72619
965824615 67064
227860495 68284
646897029 73267
373283191 72178
442244434 67973
325587033 73629
946059073 71488
110127331 71486
8808363 68021
971662165 73450
558670927 71457
200797276 75106
8838356 73887
586401706 74140
343647827 71939
121310723 72552
663870245 68583
473720719 71787
239109037 73026
441584621 71742
892498584 72565
48352161 74042
401123

In [146]:
cat /input/genomic/fish/Chrysophrys/auratus/Resequencing/CAGRF15217/sample_and_individual_identification.txt | tail -n+2 | awk '{print $3}'>cp2
cat /powerplant/workspace/hramzr/2_Phd_PROJECT/GBS/vcf_snapper_dups/SNPS_per_indv  | awk '{print $1}'>cp1
cat /powerplant/input/genomic/fish/Chrysophrys/auratus/Resequencing/CAGRF15217/parentage.txt | tail -n+2 | awk '{print $1}'>cp3

In [152]:
# cat cp2 | wc -l
grep -F -x -f cp1 cp3 | wc -l


532


In [132]:
cat /powerplant/workspace/cflasc/Snapper/GBS_3/GBS_1_keyfile.txt 

C803HANXX	8	CGAGGTGGA	662	MyPlate8	12	H	008826052ANXX	1	GGCGG	8	MyPlate1	3	H	115714093

In [4]:
ll

total 238984
-rw-rw-r--.  1 hramzr powerplant     85017 Sep  8 15:20 148842668.txt
-rw-rw-r--.  1 hramzr powerplant    150510 Sep  8 15:24 946380271.txt
-rw-rw-r--.  1 hramzr powerplant       143 Sep  7 15:34 accs
-rw-rw-r--.  1 hramzr powerplant    448405 Sep  7 15:32 chrysophrys_auratus.db.nhr
-rw-rw-r--.  1 hramzr powerplant     55032 Sep  7 15:32 chrysophrys_auratus.db.nin
-rw-rw-r--.  1 hramzr powerplant 192202795 Sep  7 15:32 chrysophrys_auratus.db.nsq
drwxrwsr-x.  9 hramzr powerplant       481 Sep 14 15:48 cnv_calling
-rw-rw-r--.  1 hramzr powerplant     52692 Sep  1 17:34 demultiplex_snapper.ipynb
-rw-rw-r--.  1 hramzr powerplant     18519 Aug 28 14:21 demultiplex_trevally.ipynb
drwxrwsr-x.  5 hramzr powerplant       226 Aug 26 11:36 Fish
drwxrwsr-x. 21 hramzr powerplant       595 Sep 11 15:44 GBS
drwxrwsr-x.  4 hramzr powerplant        58 Aug 27 11:56 Genomics
drwxrwsr-x.  4 hramzr powerplant       355 Sep  7 16:50 growth_regions
-rw-rw-r--.  1 hramzr powerplant     27728 Sep 